In [1]:

import pprint as pp
import json
import os
import pprint
import plotly.express as px
from itertools import chain
import requests
import pprint
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
import plotly.offline as off
import ipywidgets as widgets
import statistics
from datetime import datetime
import statistics


In [2]:
# starting code that loads data, run this cell, but no need to understand it in depth
# this function just loads the data from files
def load_json_file_named(file_name):
    try: 
        loaded_data = []
        file_location = f"{file_name}"
        with open(file_location, 'r') as file: # or f"data/{file_name}" depending on your files
            loaded_data =  json.load(file)
    except OSError as e:
        print(f"Error. Does the file exist in this folder? {file_location}\n\n {e}")
    return loaded_data

Loading all Json Datasets (We restrict our analysis to the first two datasets)

In [4]:

nhs_boards_info = load_json_file_named('nhs_scotland_boards.json')
covid_summary = load_json_file_named('covid_records_scotland.json')


The below function aims to exploit all values in a nested dictionary, thereby summarising it into nested key/value structures and consequently giving us a list type inference.

In [5]:
def extract_all_values(dict_of_interest, dimension):
    value_list_def = list(value for key, value in dict_of_interest.items())
    key_list_def = list(key for key, value in dict_of_interest.items())
    result = key_list_def if dimension == "key" else value_list_def
    return result

Two main classes of visualisation are intended. One to understand how the numerical data on day to day Covid-19 stats pan out across each NHS zone, and another to understand the hierarchial budget split for each of the zone. Relevant data (lists) are accrued from the respective nested dictionaries.

In [7]:



budget_count = list(chain(*list(extract_all_values(boards['budget_billions'], "values") for boards in nhs_boards_info)))

# All 14 regions
regions = list(boards['name'] for boards in nhs_boards_info)
# List of all "year" attributes
years = list(chain(*list(extract_all_values(boards['budget_billions'], "key") for boards in nhs_boards_info)))

#list indicating the number of staff for all regions
staff_count = list(extract_all_values(boards['people'], "values") for boards in nhs_boards_info)
staff_count = [list(staff) for staff in zip(*staff_count)][1]

#Extracting Area in Km2 through a list
area_in_km2 = list(extract_all_values(boards['geographic'], "values") for boards in nhs_boards_info)
area_in_km2 = [list(area) for area in zip(*area_in_km2)][0]

#Determining a staff per km2 area count
staff_per_area = [round(staff/area,2) for staff,area in zip(staff_count, area_in_km2)]

dates_covid_data = list(datetime.strptime(str(date),"%Y%m%d") for date in list(i['date'] for i in covid_summary))



 Now for the dataset comprising Covid-19 day to day stats, we plot a series of time series graphs of each of the NHS centers. Here we plot tree graphs for each center:  one for the number of new positive cases, number of new deaths, number of first infections. On the other hand, the hierarchial budget split is visualised by either a pie chart or a sunburst diagram depending on the selected region or the whole summary rather.

In [8]:


def plot_dashboard(name_nhs_center):
    number_of_new_positive = []
    number_of_new_deaths = []
    number_of_first_infections = []
    number_of_reinfections = []
    input_dictionary = covid_summary

    for part_dictionary in input_dictionary:
     if name_nhs_center == "whole_scotland" :
        fig_master = make_subplots(rows=2, cols=2, vertical_spacing=0.1, specs=[[{"type": "xy"}, {"type": "xy"}], [{"type": "xy"}, {"type": "sunburst"}]])
        number_of_new_positive.append(part_dictionary['whole_scotland']['new_positive'])
        number_of_new_deaths.append(part_dictionary['whole_scotland']['new_deaths'])
        number_of_first_infections.append(part_dictionary['whole_scotland']['first_infections'])
        number_of_reinfections.append(part_dictionary['whole_scotland']['reinfections'])


     else:
         fig_master = make_subplots(rows=2, cols=2, vertical_spacing=0.1, specs=[[{"type": "xy"}, {"type": "xy"}], [{"type": "xy"}, {"type": "pie"}]])

         if part_dictionary['locations'][name_nhs_center] == {}:
            number_of_new_positive.append(0)
            number_of_new_deaths.append(0)
            number_of_first_infections.append(0)
            number_of_reinfections.append(0)
         else:
            number_of_new_positive.append(part_dictionary['locations'][name_nhs_center]['new_positive'])
            number_of_new_deaths.append(part_dictionary['locations'][name_nhs_center]['new_deaths'])
            number_of_first_infections.append(part_dictionary['locations'][name_nhs_center]['first_infections'])
            number_of_reinfections.append(part_dictionary['locations'][name_nhs_center]['reinfections'])
    
           
    
    budget_count_specific = list(chain(*list(extract_all_values(board['budget_billions'], "values") for board in nhs_boards_info if board['name'] == name_nhs_center)))
    years_specific = list(chain(*list(extract_all_values(board['budget_billions'], "keys") for board in nhs_boards_info if board['name'] == name_nhs_center)))
    mean_new_positive = statistics.mean(number_of_new_positive)
    mean_new_deaths = statistics.mean(number_of_new_deaths)
    mean_first_infections = statistics.mean(number_of_first_infections)
    
    
    data_sunburst = dict(
    budget_count = budget_count,
    regions = list(region for region in regions for _ in range(3)) ,
    years = years,
    staff_per_area_km2 = list(staff for staff in staff_per_area for _ in range(3))
    )


    fig_sunburst = px.sunburst(data_sunburst,
                         path= ['regions', 'years'], 
                         values = 'budget_count',
                         hover_data= ['staff_per_area_km2'],
                         title = "SunBurst Chart of Accumulated Budgets over years for different NHS bases",
                        )
    fig_pi = go.Figure(data = [go.Pie(labels= years, values = budget_count_specific, textinfo = 'label', hovertext= ['budget_count'])])

    # figsun.add_trace(hovertemplate='<b>%{label} </b> <br> Sales: %{value}<br> Success rate: %{color:.2f}',)
    fig_master.add_trace(go.Scatter(x=dates_covid_data, y=number_of_new_positive, name="Number of New Positive Cases"), row=1, col=1)
    fig_master.add_trace(go.Scatter(x=dates_covid_data, y=number_of_new_deaths, name="Number Of New Deaths"), row=2, col=1)
    fig_master.add_trace(go.Scatter(x=dates_covid_data, y = number_of_first_infections, name="Number of First Infections Cases"), row=1, col=2)
    fig_master.add_hline(y=mean_new_positive, line_dash="dash", line_color="black", annotation_text=f"Mean: {mean_new_positive:.2f}", row=1, col=1)
    fig_master.add_hline(y=mean_new_deaths, line_dash="dash", line_color="black", annotation_text=f"Mean: {mean_new_deaths:.2f}", row=2, col=1)
    fig_master.add_hline(y=mean_first_infections, line_dash="dash", line_color="black", annotation_text=f"Mean: {mean_first_infections:.2f}", row=1, col=2)

    if name_nhs_center == "whole_scotland" :
       fig_master.add_trace(fig_sunburst.data[0],row=2,col=2)
       fig_master.update_traces(customdata = list(staff_per_area*4) ,hovertemplate='<b>%{label} </b> <br> Staff per square km: %{customdata}<br> Budget in billions : %{value}', row = 2, col= 2)
       fig_master.update_layout(height=1200, width=1500, title_text= "Time series graphs and Sunburst chart of Budget distribution over years and NHS zones for the Whole of Scotland")

       

    else:   
       fig_master.add_trace(fig_pi.data[0],row=2,col=2)
       fig_master.update_traces(hovertemplate='<br> Budget in billions : %{value}', row = 2, col= 2)
       fig_master.update_layout(height=1200, width=1500, title_text= " Time series graphs and Pie chart of Budget distribution over years for: " + str(name_nhs_center))

       
   
    
    fig_master.update_yaxes(title_text="Count of new positive Cases", row=1, col=1)
    fig_master.update_yaxes(title_text="Count of Deaths due to Covid", row=2, col=1)
    fig_master.update_yaxes(title_text="Count of Covid-19 First infections", row=1, col=2)
    fig_master.update_coloraxes(title_text="Budget Split over years and NHS Zones", row=2, col=2)
    
    fig_master.update_xaxes(title_text="Date", row=1, col=1)
    fig_master.update_xaxes(title_text="Date", row=2, col=1)
    fig_master.update_xaxes(title_text="Date", row=1, col=2)
    fig_master.show()

These commands ensure a dropdown widget is initialised, with options to filter the area (NHS zone) of interest. Owing to the notebook format, graphs load a bit sluggish!!

In [9]:
dropdown_values = list(covid_summary[0].keys())[1:2]
dropdown_values.extend(covid_summary[0]['locations'].keys())
widgets.interact(plot_dashboard, name_nhs_center=widgets.Dropdown(options=dropdown_values, description = "Select Area of interest"))

interactive(children=(Dropdown(description='Select Area of interest', options=('whole_scotland', 'NHS Ayrshire…

<function __main__.plot_dashboard(name_nhs_center)>

So here are the takeaways!! We focus on the spate of the Whole of Scotland with other inferences seemingly following the same suite. From the first time series plot, we observe a peak on the 29th of December 2021, with over 23500 recorded cases with a worrying 21500 first time infections. The difference indicates a considerable number of reinfections as well. The peak of the death counts is  at the maximum at January 16th 2021 rather (76) deaths. This date however sees only 1800 additional cases, with the high death rate seemingly attributed to panic hospitalisations, and a lack of pandemic awareness. The situation on both the number of infections and deaths seem to ease around March 2022, indicating herd immunisation and the effects of proactive measures like large scale vaccinations.

Coming to the sunburst plot, we see that across all NHS zones, there is a considerable budget increase from 2020 to 2022. This can be one such potential cause in reducing both the effects of COvid-19 as well as any further comorbidites associated. Figuratively, speaking from the words of Peter "Littlefinger" Bailish, "Chaos is a Ladder":. Desperate times called for desperate budgets and thankfully, we can assume that winter has arrived for House COViD.